In [1]:
import sys
sys.path.append("D:\\WETrak")

import torch.nn as nn

In [2]:
class MLP_regression(nn.Module):
    def __init__(self, emg_channel=5, hidden_size=64, output_size=2*4):
        super(MLP_regression, self).__init__()
        self.emg_channel = emg_channel
        
        self.layer1 = nn.Sequential(nn.Linear(1000*emg_channel, hidden_size), 
                                   nn.ReLU(True), 
                                   nn.Dropout(p=0.05))
        
        self.layer2 = nn.Sequential(nn.Linear(hidden_size, hidden_size), 
                                   nn.ReLU(True), 
                                   nn.Dropout(p=0.05))
        
        self.layer3 = nn.Sequential(nn.Linear(hidden_size, output_size), 
                                   nn.Sigmoid(), 
                                   nn.Dropout(p=0.05))
        
        # initialize weights
        self._initialize_weights()

    def forward(self, x):
        # Flatten the input tensor from [Batch, 1000, 2] to [Batch, 2000]
        x = x.view(-1, self.emg_channel * 1000)
        # Apply fully connected layers
        x = self.layer1(x)
        x = self.layer2(x)
        x = self.layer2(x)
        x = self.layer3(x)

        return x
    
    
    def _initialize_weights(self):
        for m in self.modules():
            if isinstance(m, nn.Conv2d):
                nn.init.kaiming_normal_(m.weight, mode='fan_out', nonlinearity='relu')
                if m.bias is not None:
                    nn.init.constant_(m.bias, 0)
            elif isinstance(m, nn.BatchNorm2d):
                nn.init.constant_(m.weight, 1)
                nn.init.constant_(m.bias, 0)
            elif isinstance(m, nn.Linear):
                nn.init.normal_(m.weight, 0, 0.01)
                nn.init.constant_(m.bias, 0)